In [52]:
import pandas as pd
import numpy as np
file_directory='D:\\chengjunyi\\乌鲁木齐最终实验\\随机森林实验\\数据集\\'
file_name1='train set-multiscale.csv'
data_train=pd.read_csv(file_directory+file_name1,engine='python')
file_name2='test set-multiscale.csv'
data_test=pd.read_csv(file_directory+file_name2,engine='python')
file_name3='validation set-multiscale.csv'
data_valid=pd.read_csv(file_directory+file_name3,engine='python')
data_train.describe()

,Y,tid,age25_250,age25_onegrid,age25_threegrid,age25_twogrid,age35_250,age35_onegrid,age35_threegrid,age35_twogrid,...,娱乐场所j1000,娱乐场所j1500,娱乐场所j250,娱乐场所j500,娱乐场所j750,住宅区j1000,住宅区j1500,住宅区j250,住宅区j500,住宅区j750
count,5540.000000,5540.000000,5540.000000,5540.000000,5540.000000,5540.000000,5540.000000,5540.000000,5540.000000,5540.000000,...,5540.000000,5540.000000,5540.000000,5540.000000,5540.000000,5540.000000,5540.000000,5540.000000,5540.000000,5540.000000
mean,1.340195,7817.636282,0.298106,0.289767,0.285504,0.286902,0.174921,0.168472,0.167947,0.168279,...,11.101083,22.212274,1.368953,3.620758,6.868231,53.851083,109.151986,6.585740,17.428339,33.211733
std,2.990523,3668.030246,0.227178,0.105502,0.092502,0.090210,0.188037,0.073433,0.073897,0.068164,...,21.973557,41.284565,3.853563,8.436997,14.424031,86.591956,169.368852,12.203237,29.898065,54.727012
min,0.000000,70.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,4907.750000,0.176471,0.239247,0.248207,0.246262,0.000000,0.135467,0.145568,0.143294,...,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,5.000000,0.000000,0.000000,0.000000
50%,0.000505,8165.500000,0.280502,0.288959,0.287850,0.288926,0.155340,0.169575,0.171122,0.171445,...,0.000000,2.000000,0.000000,0.000000,0.000000,9.000000,21.000000,0.000000,2.000000,5.000000
75%,1.180362,10721.250000,0.377869,0.341093,0.327104,0.332113,0.222222,0.201717,0.191087,0.193425,...,11.000000,24.000000,0.000000,2.000000,6.000000,68.000000,144.000000,7.000000,20.000000,40.250000
max,24.107420,13868.000000,1.166667,1.111111,1.000000,1.000000,1.166667,1.000000,1.166667,1.166667,...,126.000000,219.000000,49.000000,80.000000,96.000000,401.000000,766.000000,105.000000,173.000000,279.000000


In [53]:
X_train=data_train.iloc[:,2:52]
Y_train=data_train.iloc[:,0]
X_test=data_test.iloc[:,2:52]
Y_test=data_test.iloc[:,0]
X_valid=data_valid.iloc[:,2:52]
Y_valid=data_valid.iloc[:,0]

In [38]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签
plt.rcParams['axes.unicode_minus']=False #用来正常显示负号
%matplotlib inline
sns.set_style("whitegrid")
sns.set(rc = {'figure.figsize':(100, 100)})
def imp_df(column_names, importances):
    df = pd.DataFrame({'feature': column_names,
                       'feature_importance': importances}) \
           .sort_values('feature_importance', ascending = False) \
           .reset_index(drop = True)
    return df

# plotting a feature importance dataframe (horizontal barchart)
def var_imp_plot(imp_df, title):
    imp_df.columns = ['feature', 'feature_importance']
    sns.barplot(x = 'feature_importance', y = 'feature', data = imp_df, orient = 'h', color = 'royalblue') \
       .set_title(title, fontsize = 12)
    

In [39]:
def IncMSE(MSE,x_test, y_test,FeatureNum,Set_Times,model,VI):    #获取MSE，x测试集，y测试集，特征数，随机求IncMSE次数，模型（随机森林）
    x_MSE = copy.deepcopy(x_test)      #深拷贝不破坏原列表
    y_MSE = copy.deepcopy(y_test)
    TestNum = len(y_MSE)
    #########多次生成随机数，多次计算IncMSE（由于随机有不确定性，所以要多次随机）
    IncMSE_Set = []
    IncMSE_Times = 1
    while IncMSE_Times <= Set_Times:     #多次生成随机数，多次计算IncMSE（由于随机有不确定性，所以要多次随机）
        IncMSE_x = []
        for i in range(0,FeatureNum):
            MSE_Replace = np.random.random(TestNum)
            x_MSE[:,i] = MSE_Replace           #替换第i个特征
            MSE_Score = model.score(x_MSE,y_MSE)
            MSE_Result = model.predict(x_MSE)
            MSE_ResidualSquare = (MSE_Result - y_MSE)**2   #计算残差平方
            MSE_RSS = sum(MSE_ResidualSquare)   #计算残差平方和
            MSE_MSE = np.mean(MSE_ResidualSquare)   #计算均方差
            IncMSE = MSE_MSE - MSE
            IncMSE_x.append(IncMSE)
            x_MSE = copy.deepcopy(x_test)   #复原原特征，深拷贝不破坏原列表            
        IncMSE_Set.append(IncMSE_x)          #多次计算IncMSE后的数据
        
        IncMSE_Times += 1
    IncMSE_SetArray = np.array(IncMSE_Set)    #变换为array
    ########计算每个特征的IncMSE平均数########
    X_IncMSE_Average = []
    for j in range(0,FeatureNum):
        X_IncMSE_Set = IncMSE_SetArray[:,j]
        X_IncMSE = np.mean(X_IncMSE_Set)       #求多次IncMSE的平均值（由于随机有不确定性，所以要多次随机）
        X_IncMSE_Average.append(X_IncMSE)
        VI.append(X_IncMSE)
    X_IncMSE_Average_Sum = sum(X_IncMSE_Average)
    ########计算每个特征的IncMSE平均数的百分比########
    print('IncMSE:')
    for k in range(0,FeatureNum):
        X_Percent = X_IncMSE_Average[k]/X_IncMSE_Average_Sum       #计算每个特征IncMSE的百分比
        print(f'    x{k+1} = {X_IncMSE_Average[k]}   {X_Percent*100}%')        #输出各特征的IncMSE的平均数与其百分比

In [40]:
from sklearn.ensemble import RandomForestRegressor
from sklearn import ensemble
import copy


In [50]:
model = RandomForestRegressor(n_estimators = 200,
                               n_jobs = -1,max_features=8,max_depth=10,
                               oob_score = True,
                               bootstrap = True,
                               random_state = 42)
model.fit(X_train, Y_train)

score = model.score(X_valid, Y_valid)
result = model.predict(X_valid)
ResidualSquare = (result - Y_valid)**2     #计算残差平方
RSS = sum(ResidualSquare)   #计算残差平方和
MSE = np.mean(ResidualSquare)      #计算均方差
RMSE = np.mean(ResidualSquare)**0.5      #计算均方差
num_regress = len(result)   #回归样本个数


VI=[]

x_valid1=X_valid.values
y_valid1=Y_valid.values
IncMSE(MSE,x_valid1,y_valid1,50,20,model,VI)

IncMSE:
    x1 = -0.012405807279139324   -0.7652209160234137%
    x2 = -0.03629424365471399   -2.2387188315054996%
    x3 = 0.011674450819064052   0.7201090383579025%
    x4 = -0.007191962294946652   -0.44361804528425186%
    x5 = -0.0071962616697121675   -0.4438832413672127%
    x6 = -0.019137927903890205   -1.180474788289708%
    x7 = -0.019589137180519724   -1.2083064938943315%
    x8 = -0.032471941224375064   -2.002949751650392%
    x9 = 0.04851557606480002   2.9925608807527353%
    x10 = 0.0776006858755547   4.786602483305453%
    x11 = 0.006054795138803182   0.37347475889295967%
    x12 = 0.007260911527694258   0.4478710047132568%
    x13 = 0.025389989153203472   1.5661174094094759%
    x14 = 0.001962887325995366   0.12107575136804237%
    x15 = 0.008831004430859346   0.5447182233237013%
    x16 = 0.03407397323828852   2.1017670537091697%
    x17 = 0.019007418831323286   1.172424665487385%
    x18 = 0.00355878445269292   0.21951463839026283%
    x19 = 0.011845051398086314   0.730

In [54]:
model = RandomForestRegressor(n_estimators = 200,
                               n_jobs = -1,max_features=8,max_depth=10,
                               oob_score = True,
                               bootstrap = True,
                               random_state = 42)
model.fit(X_train, Y_train)

score = model.score(X_test, Y_test)
result = model.predict(X_test)
ResidualSquare = (result - Y_test)**2     #计算残差平方
RSS = sum(ResidualSquare)   #计算残差平方和
MSE = np.mean(ResidualSquare)      #计算均方差
RMSE = np.mean(ResidualSquare)**0.5      #计算均方差
num_regress = len(result)   #回归样本个数


VI=[]

x_valid2=X_test.values
y_valid2=Y_test.values
IncMSE(MSE,x_valid2,y_valid2,50,20,model,VI)

IncMSE:
    x1 = -0.007767525850756407   -0.5039509986583496%
    x2 = -0.04049624600594465   -2.627364750208414%
    x3 = 0.01204682017126162   0.7815882653771462%
    x4 = -0.02186283048238774   -1.418443332766583%
    x5 = 0.0031828746663776   0.20650241757086044%
    x6 = 0.022044559067640135   1.4302337411646182%
    x7 = 0.0666218551636513   4.322373827558555%
    x8 = 0.0329917390839243   2.1404782138189185%
    x9 = 0.04251274532328821   2.7581936478901907%
    x10 = 0.09157732909555862   5.941465447157402%
    x11 = 0.007518094665114483   0.48776809852824654%
    x12 = 0.03118989992218372   2.0235762990486625%
    x13 = 0.05572634050263092   3.6154813627226368%
    x14 = 0.004560182482573949   0.2958610708625658%
    x15 = 0.004498700197733718   0.2918721483355794%
    x16 = 0.05577731089067024   3.618788281611348%
    x17 = 0.018605576278093505   1.2071152289819866%
    x18 = 0.0009382859699694768   0.060875259468520404%
    x19 = 0.002160422189589062   0.14016650100508427%
 

In [11]:
file_directory='D:\\chengjunyi\\乌鲁木齐最终实验\\随机森林实验\\数据集\\'
file_name1='train set-optimal scale.csv'
data_train=pd.read_csv(file_directory+file_name1,engine='python')
file_name2='test set-optimal scale.csv'
data_test=pd.read_csv(file_directory+file_name2,engine='python')
file_name3='validation set-optimal scale.csv'
data_valid=pd.read_csv(file_directory+file_name3,engine='python')
data_train.describe()

,Y,age35_twogrid,local_threegrid,people_threegrid,rich1_threegrid,路网可达性,网吧j1500,娱乐场所j1000,住宅区j1000,atmj1500,医院j1500,non_local_twogrid,age25_threegrid
count,5540.000000,5540.000000,5540.000000,5540.000000,5540.000000,5540.000000,5540.000000,5540.000000,5540.000000,5540.000000,5540.000000,5540.000000,5540.000000
mean,1.340195,0.168279,0.445181,16918.912274,0.000034,31.355041,6.362816,11.101083,53.851083,39.151264,19.590614,0.511357,0.285504
std,2.990523,0.068164,0.179573,22730.450618,0.000123,13.385704,10.644010,21.973557,86.591956,72.484831,40.963049,0.185766,0.092502
min,0.000000,0.000000,0.000000,3.000000,0.000000,2.800081,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.143294,0.313309,1185.750000,0.000000,20.496530,0.000000,0.000000,2.000000,0.000000,0.000000,0.366383,0.248207
50%,0.000505,0.171445,0.441925,4884.000000,0.000000,30.538046,1.000000,0.000000,9.000000,5.000000,2.000000,0.506084,0.287850
75%,1.180362,0.193425,0.577606,25195.000000,0.000000,40.453534,9.000000,11.000000,68.000000,39.000000,16.000000,0.647510,0.327104
max,24.107420,1.166667,1.000000,271068.000000,0.001530,96.117348,56.000000,126.000000,401.000000,424.000000,221.000000,1.000000,1.000000


In [12]:
X_train=data_train.iloc[:,1:13]
Y_train=data_train.iloc[:,0]
X_test=data_test.iloc[:,1:13]
Y_test=data_test.iloc[:,0]
X_valid=data_valid.iloc[:,1:13]
Y_valid=data_valid.iloc[:,0]
from sklearn.ensemble import RandomForestRegressor
from sklearn import ensemble

In [17]:
model = RandomForestRegressor(n_estimators = 40,
                               max_features=4,max_depth=10, random_state=42, bootstrap=True,  n_jobs=-1)
model.fit(X_train, Y_train)

score = model.score(X_test, Y_test)
result = model.predict(X_test)
ResidualSquare = (result - Y_test)**2     #计算残差平方
RSS = sum(ResidualSquare)   #计算残差平方和
MSE = np.mean(ResidualSquare)      #计算均方差
RMSE = np.mean(ResidualSquare)**0.5      #计算均方差
num_regress = len(result)   #回归样本个数
print(f'n={num_regress}')
print(f'R^2={score}')
print(f'RMSE={RMSE}')
print(f'RSS={RSS}')

n=2750
R^2=0.8254369670576089
RMSE=1.1549759115044664
RSS=3668.415729427826


In [18]:
base_imp = imp_df(X_train.columns, model.feature_importances_)
base_imp

,feature,feature_importance
0,网吧j1500,0.278086
1,娱乐场所j1000,0.171796
2,people_threegrid,0.147511
3,住宅区j1000,0.106260
4,rich1_threegrid,0.080599
5,路网可达性,0.060348
6,医院j1500,0.048776
7,atmj1500,0.033027
8,non_local_twogrid,0.026960
9,local_threegrid,0.023491
